In [1]:
import pyreadr
import math
import numpy as np
import pandas as pd
from pymatreader import read_mat
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib
import matplotlib.pyplot as plt
import time

In [2]:
def recursionStart(spectra):
    featurePairs = []
    maxima = []
    minima = []
    
    for i in range(1, len(spectra) - 1):
        if spectra[i] > spectra[i-1]:
            if spectra[i] >= spectra[i+1]:
                for j in range(i+1, len(spectra)):
                    if spectra[i] > spectra[j]:
                        maxima.append([i, spectra[i]])
                        break
                    elif spectra[i] < spectra[j]:
                        break
        if spectra[i] < spectra[i-1]:
            if spectra[i] <= spectra[i+1]:
                for j in range(i+1, len(spectra)):
                    if spectra[i] < spectra[j]:
                        minima.append([i, spectra[i]])
                        break
                    elif spectra[i] > spectra[j]:
                        break
    maxima.sort(key = lambda element: element[1], reverse = True)
    minima.sort(key = lambda element: element[1])
    
    globalMaxima = maxima.pop(0)
    featurePairs.append([globalMaxima[0], globalMaxima[1] - minima[0][1]])
    
    overall_peaks = len(maxima)
    
    recursionStep(0, globalMaxima[0], maxima.copy(), minima.copy(), featurePairs, overall_peaks)
    recursionStep(len(spectra) - 1, globalMaxima[0], maxima.copy(), minima.copy(), featurePairs, overall_peaks)
    
    return featurePairs

def recursionStep(start, end, maxima, minima, featurePairs, overall_peaks):
    overall_peaks = overall_peaks - 1
    if overall_peaks % 1000 == 0:
        print('{} peaks are left'.format(overall_peaks))
    factor = 1
    if end < start:
        factor = -1
    currentMaxima = []
    for i in range(len(maxima)):
        position = maxima[i][0]
        if start * factor < position * factor and position * factor < end * factor:
            currentMaxima.append(maxima[i])
    if len(currentMaxima) == 0:
        return
    localMaxima = currentMaxima.pop(0)
    recursionStep(start, localMaxima[0], currentMaxima.copy(), minima.copy(), featurePairs, overall_peaks)
    currentMinima = []
    for i in range(len(minima)):
        position = minima[i][0]
        if localMaxima[0] * factor < position * factor and position * factor < end * factor:
            currentMinima.append(minima[i])
    localMinima = currentMinima.pop(0)
    featurePairs.append([localMaxima[0], localMaxima[1] - localMinima[1]])
    recursionStep(localMinima[0], localMaxima[0], currentMaxima.copy(), currentMinima.copy(), featurePairs, overall_peaks)
    recursionStep(localMinima[0], end, currentMaxima.copy(), currentMinima.copy(), featurePairs, overall_peaks)

In [3]:
def getPersistenceTransformation(data_X):
    spectras = []
    for i in range(len(data_X)):
        featurePairs = recursionStart(data_X.iloc[i])
        featurePairs.sort(key = lambda element: element[1], reverse = True)
        spectras.append(featurePairs)
    
    return spectras
   
def getPercantageForSpectra(spectras, length, k):
    transformationForSpectra = []
    for i in range(0, len(spectras)):
        transformation = [0] * length
        featurePairs = spectras[i][0:round(k*len(spectras[i]))]
        for x, fx in featurePairs:
            transformation[x] = fx
        transformationForSpectra.append(transformation)
    
    return transformationForSpectra

In [4]:
def cvAccuracy(tmas, X, resp, classifier, ntrees = 150):
    
    """
    'Object: This functions aims to execute a 8-fold cross validation based on each TMA.'
    'Input: tmas, X refers to spectral data, resp is the response variables.'
    'The use needs to select which classifier wants to use. Number of trees is the only RF parameter tha one can select'
    """
    
    tmas_names = ['TMA_1:', 'TMA_2:', 'TMA_3:', 'TMA_4:', 'TMA_5:', 'TMA_6:', 'TMA_7:', 'TMA_8:']
    
    tmas_res_test   = []
    tmas_vals_test  = []
    tmas_res_train  = []
    tmas_vals_train = []
    
    
    mtry =  round(math.sqrt(X.shape[1]))
    
    for tma in range(1, len(tmas_names)+1):
    
        X_train = X.iloc[tmas[tmas != tma].index, :]

        X_test = X.iloc[tmas[tmas == tma].index, :]

        y_train = resp.iloc[tmas[tmas != tma].index, :]

        y_test = resp.iloc[tmas[tmas == tma].index, :]
        
        if (classifier == 'logit'):

            np.random.seed(1234)
            
            logreg = LogisticRegression(penalty = None, solver = 'newton-cg',fit_intercept=True, random_state = 1234)

            logreg.fit(X_train, y_train.values.ravel())

            y_pred = logreg.predict_proba(X_test)

            y_pred_1 = np.where(y_pred[:, 1] > 0.5, 1, 0)

            accuracy = balanced_accuracy_score(y_test, y_pred_1)
            
            tmas_vals_test.append(accuracy)

            tmp_tmas = [tmas_names[tma-1], accuracy]

            tmas_res_test.append(tmp_tmas)
            
        if (classifier == 'rf'):
            
            'Set up accordingly the default values as reported in Probst et al. 2019'
            
            rf = RandomForestClassifier(n_estimators= ntrees,
                                        random_state= 1234, 
                                        criterion = 'gini',
                                        max_features= mtry,
                                        bootstrap=True,
                                        min_samples_leaf=1,
                                        n_jobs = -3)
            
            rf.fit(X_train, y_train.values.ravel())
            
            y_pred_rf = rf.predict(X_test)
            
            accuracy = balanced_accuracy_score(y_test, y_pred_rf)
            
            tmas_vals_test.append(accuracy)

            tmp_tmas = [tmas_names[tma-1], accuracy]

            tmas_res_test.append(tmp_tmas)
        
    return tmas_vals_test, tmas_res_test  

def Average(lst):
    return sum(lst) / len(lst)   

In [5]:
# Loading Data from Matlab. In the subfolder 'data', we provided a matlab code, how one can extract the 
## dataset from the paper by Leuschner  el at (2019) 
dataMaldi = read_mat('data/L1-8_tic_ad_sq.mat')

X_values = pd.DataFrame(dataMaldi['data_tic']) ## This object relates to the pre-processed data
                                
classes = pd.DataFrame(dataMaldi['classes']) #The cancer subtpyes

mz_values = pd.DataFrame(dataMaldi['mzVector']) ### The m/z vector

TMAs   = pd.Series(dataMaldi['tmas']) 

p = X_values.shape[1]

y = pd.DataFrame(np.where(classes == 1, 0, 1))

In [6]:
#This code uses persistence transformation and returns all the persistence values
st = time.time()
spectras = getPersistenceTransformation(data_X = X_values)
et = time.time()
print('Time for the processing: ', et-st)

Time for the processing:  270.592161655426


In [7]:
# We experiment over a grid of levels of peaks extraction. Namely, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5
X_k0 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.01)
X_k1 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.05)
X_k2 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.1)
X_k3 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.2)
X_k4 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.25)
X_k5 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.3)
X_k6 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.4)
X_k7 = getPercantageForSpectra(spectras, len(X_values.iloc[0]), 0.5)
spectrasForEachK = 0
listOfk = [0.01, 0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5]

In [19]:
'''This chunk of code executes logit-based classification. It usually takes around 10 minutes on a standard machine '''
q1, q2 = cvAccuracy(tmas =TMAs, resp=y, X = X_values, classifier='logit')
res_k0_1, res_k0_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k0), classifier = 'logit')
res_k1_1, res_k1_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k1), classifier = 'logit')
res_k2_1, res_k2_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k2), classifier = 'logit')
res_k3_1, res_k3_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k3), classifier = 'logit')
res_k4_1, res_k4_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k4), classifier = 'logit')
res_k5_1, res_k5_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k5), classifier = 'logit')
res_k6_1, res_k6_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k6), classifier = 'logit')
res_k7_1, res_k7_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k7), classifier = 'logit')

In [21]:
results_as_tables = pd.concat([pd.DataFrame(res_k0_1).describe(), pd.DataFrame(res_k1_1).describe(),
                              pd.DataFrame(res_k2_1).describe(), pd.DataFrame(res_k3_1).describe(),
                              pd.DataFrame(res_k4_1).describe(), pd.DataFrame(res_k5_1).describe(),
                              pd.DataFrame(res_k6_1).describe(), pd.DataFrame(res_k7_1).describe(),
                               pd.DataFrame(q1).describe()],
                              axis=1)
results_to_latex = round(results_as_tables, 3)

results_to_latex = results_to_latex.set_axis(['k = 0.01', 'k = 0.05', ' k = 0.1', 'k = 0.2', 
                           'k = 0.25', 'k = 0.3', 'k = 0.4', 'k = 0.5', 'raw'], axis=1)

ValueError: Length mismatch: Expected axis has 8 elements, new values have 9 elements

In [ ]:
results_to_latex

In [8]:
st0 = time.time()
rf_k0_1000, rf_k0_2_1000= cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k0), classifier = 'rf', ntrees = 1000)
et0 = time.time()
print('Time for processing ',  ': ', et0 - st0)
st1 = time.time()
rf_k1_1000, rf_k1_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k1), classifier = 'rf', ntrees = 1000)
et1 = time.time()
print('Time for processing ',  ': ', et1 - st1)
st2 = time.time()
rf_k2_1000, rf_k2_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k2), classifier = 'rf', ntrees = 1000)
et2 = time.time()
print('Time for processing ',  ': ', et2 - st2)
st3 = time.time()
rf_k3_1000, rf_k3_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k3), classifier = 'rf', ntrees = 1000)
et3 = time.time()
print('Time for processing ',': ', et3 - st3)
st4 = time.time()
rf_k4_1000, rf_k4_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k4), classifier = 'rf', ntrees = 1000)
et4 = time.time()
print('Time for processing ', ': ', et4 - st4)
st5 = time.time()
rf_k5_1000, rf_k5_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k5), classifier = 'rf', ntrees = 1000)
et5 = time.time()
print('Time for processing ', ': ', et5 - st5)
st6 = time.time()
rf_k6_1000, rf_k6_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k6), classifier = 'rf', ntrees = 1000)
et6 = time.time()
print('Time for processing ', ': ', et6 - st6)
st7 = time.time()
rf_k7_1000, rf_k7_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k7), classifier = 'rf', ntrees = 1000)
et7 = time.time()
print('Time for processing ', ': ', et7 - st7)
st8 = time.time()
rf_raw_1000, rf_raw_2_1000 = cvAccuracy(tmas =TMAs, resp=y, X = X_values, classifier='rf', ntrees = 1000)
et8 = time.time()
print('Time for processing the raw data: ', et8 - st8)

Time for processing  :  113.76704216003418
Time for processing  :  85.07751679420471
Time for processing  :  101.20961356163025
Time for processing  :  129.37617874145508
Time for processing  :  141.56318402290344
Time for processing  :  158.50366139411926
Time for processing  :  187.1691517829895
Time for processing  :  215.02939319610596
Time for processing the raw data:  799.5551612377167


In [9]:
results_as_tables_rf_1000 = pd.concat([pd.DataFrame(rf_k0_1000).describe(),
                              pd.DataFrame(rf_k1_1000).describe(), pd.DataFrame(rf_k2_1000).describe(),
                              pd.DataFrame(rf_k3_1000).describe(), pd.DataFrame(rf_k4_1000).describe(),
                              pd.DataFrame(rf_k5_1000).describe(), pd.DataFrame(rf_k6_1000).describe(), 
                              pd.DataFrame(rf_k7_1000).describe(), pd.DataFrame(rf_raw_1000).describe()],
                              axis=1)

results_rf_to_latex_1000 = round(results_as_tables_rf_1000, 3)

results_rf_to_latex_1000 = results_rf_to_latex_1000.set_axis(['k = 0.01', 'k = 0.05', ' k = 0.1', 'k = 0.2', 
                           'k = 0.25', 'k = 0.3' ,'k = 0.4', 'k = 0.5', 'raw'], axis=1)

pd.DataFrame(results_rf_to_latex_1000).style.to_latex('results/results_rf_to_latex.tex')

In [22]:
results_rf_to_latex_1000

,k = 0.01,k = 0.05,k = 0.1,k = 0.2,k = 0.25,k = 0.3,k = 0.4,k = 0.5,raw
count,8.000,8.000,8.000,8.000,8.000,8.000,8.000,8.000,8.000
mean,0.643,0.793,0.831,0.863,0.871,0.878,0.877,0.868,0.873
std,0.108,0.086,0.083,0.076,0.066,0.067,0.061,0.079,0.062
min,0.514,0.646,0.702,0.734,0.760,0.774,0.776,0.746,0.775
25%,0.575,0.727,0.777,0.834,0.851,0.851,0.854,0.835,0.837
50%,0.622,0.826,0.849,0.873,0.880,0.892,0.900,0.890,0.879
75%,0.708,0.852,0.878,0.916,0.919,0.925,0.920,0.924,0.923
max,0.840,0.890,0.936,0.945,0.940,0.959,0.930,0.949,0.943


In [17]:
'''CV 2 '''
def cv2(tmas, X, resp, classifier, validation_tma, ntrees = 150):
    
    """
    'Object: This functions aims to execute a 2-fold cross validation based on each TMA.'
    'Input: tmas, X refers to spectral data, resp is the response variable.'
    'The use needs to select which classifier wants to use. Number of trees is the only RF parameter tha one can select'
    """
    
    tmas_res_test   = []
    tmas_vals_test  = []
    tmas_res_train  = []
    tmas_vals_train = []
    
    mtry =  round(math.sqrt(X.shape[1]))
    
    for i in range(0, len(validation_tma)):
      
        if i == 0:
            
            X_train = X.iloc[tmas[(tmas >= validation_tma[i][0]) & (tmas <= validation_tma[i][3])].index, :]

            X_test =  X.iloc[tmas[(tmas >= validation_tma[1][0]) & (tmas <= validation_tma[1][3])].index, :]

            y_train = resp.iloc[tmas[(tmas >= validation_tma[i][0]) & (tmas <= validation_tma[i][3])].index, :]

            y_test =  resp.iloc[tmas[(tmas >= validation_tma[1][0]) & (tmas <= validation_tma[1][3])].index, :]

        elif i == 1:

            X_train = X.iloc[TMAs[(TMAs >= validation_tma[i][0]) & (TMAs <= validation_tma[i][3])].index, :]
        
            X_test =  X.iloc[TMAs[(TMAs < validation_tma[i][0]) & (TMAs < validation_tma[i][3])].index, :]
        
            y_train = resp.iloc[TMAs[(TMAs >= validation_tma[i][0]) & (TMAs <= validation_tma[i][3])].index, :]

            y_test =  resp.iloc[TMAs[(TMAs < validation_tma[i][0]) & (TMAs < validation_tma[i][3])].index, :]
            
        if (classifier == 'logit'):

            np.random.seed(1234)

            logreg = LogisticRegression(penalty = None, solver = 'newton-cg',fit_intercept=True, random_state = 1234)

            logreg.fit(X_train, y_train.values.ravel())

            y_pred = logreg.predict_proba(X_test)

            y_pred_1 = np.where(y_pred[:, 1] > 0.5, 1, 0)

            accuracy = balanced_accuracy_score(y_test, y_pred_1)

            tmas_vals_test.append(accuracy)

            tmp_tmas = [tmas_names[tma-1], accuracy]

            tmas_res_test.append(tmp_tmas)

        elif (classifier == 'rf'):

            'Set up accordingly the default values as reported in Probst et al. 2019'

            rf = RandomForestClassifier(n_estimators= ntrees,
                                            random_state= 1234, 
                                            criterion = 'gini',
                                            max_features= mtry,
                                            bootstrap=True,
                                            min_samples_leaf=1,
                                            n_jobs = -3)

            rf.fit(X_train, y_train.values.ravel())

            y_pred_rf = rf.predict(X_test)

            accuracy = balanced_accuracy_score(y_test, y_pred_rf)

            tmas_vals_test.append(accuracy)

    return tmas_vals_test  

def Average(lst):
    return sum(lst) / len(lst)   

In [24]:
results_cv2_x4 = cv2(tmas =TMAs, resp=y, X = pd.DataFrame(X_k4), classifier = 'rf',  validation_tma = [[1, 2, 3, 4], [5, 6, 7, 8]], ntrees = 1000)
print('Results for 2-Fold CV with {:.2f}%: {}'.format(listOfk[4] * 100, results_cv2_x4))
results_cv2_x5 = cv2(tmas =TMAs, resp=y, X = pd.DataFrame(X_k5), classifier = 'rf',  validation_tma = [[1, 2, 3, 4], [5, 6, 7, 8]], ntrees = 1000)
print('Results for 2-Fold CV with {:.2f}%: {}'.format(listOfk[5] * 100, results_cv2_x5))
results_cv2_x6 = cv2(tmas =TMAs, resp=y, X = pd.DataFrame(X_k6), classifier = 'rf',  validation_tma = [[1, 2, 3, 4], [5, 6, 7, 8]], ntrees = 1000)
print('Results for 2-Fold CV with {:.2f}%: {}'.format(listOfk[6] * 100, results_cv2_x6))

Results for 2-Fold CV with 25.00%: [0.8449038563389484, 0.9022338318808417]
Results for 2-Fold CV with 30.00%: [0.8440220116724975, 0.9075487181858111]
Results for 2-Fold CV with 40.00%: [0.8353168974178729, 0.9149976498146821]


In [ ]:
matplotlib.style.use('ggplot')

#[0.01, 0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5]
fig = plt.figure(figsize=(3, 3), dpi=300)

#plt.subplot(1, 2, 1)

s1 = X_values.loc[[22]]
s2 = pd.DataFrame(X_k3).loc[[22]]
s3 = pd.DataFrame(X_k4).loc[[22]]
s4 = pd.DataFrame(X_k5).loc[[22]]

fig = plt.figure()

fig, ax = plt.subplots(4, 1)


ax[0].plot(mz_values, np.transpose(s1))

ax[0].ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)
ax[0].set_ylabel('Intensity')
ax[0].set_xlabel('m/z values')
ax[0].set_title("raw spectrum")

#.subplot(2, 2, 2) # index 2
ax[1].plot(mz_values, np.transpose(s2))
ax[1].ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)
ax[1].set_ylabel('Persistance')
ax[1].set_xlabel('m/z values')
ax[1].set_title("k = 20%")

ax[2].plot(mz_values, np.transpose(s3))
ax[2].ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)
ax[2].set_ylabel('Persistance')
ax[2].set_xlabel('m/z values')
ax[2].set_title("k = 25%")

ax[3].plot(mz_values, np.transpose(s4))
ax[3].ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)
ax[3].set_ylabel('Persistance')
ax[3].set_xlabel('m/z values')
ax[3].set_title("k = 30%")

plt.subplots_adjust(left=0.1,
                    bottom=0.01,
                    right=1.5,
                    top=2,
                    wspace=0.6,
                    hspace=0.7)

#plt.savefig("results/Figure6.pdf", format="pdf", bbox_inches="tight")



In [ ]:
import matplotlib
matplotlib.style.use('ggplot')
s1 = X_values.loc[[22]]

fig = plt.figure(figsize=(5,5), dpi=500)

plt.subplot(1, 2, 1)
plt.plot(mz_values, np.transpose( X_values.loc[[22]]))
plt.ylabel('Intensity')
plt.xlabel('m/z values')

plt.ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)

plt.subplot(1, 2, 2) # index 2
plt.plot(mz_values[250:260], np.transpose( X_values.loc[[22]])[250:260], marker = 'o')
plt.ylabel('Intensity')
plt.xlabel('m/z values')


plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=1.5,
                    top=0.85,
                    wspace=0.6,
                    hspace=0.4)

plt.ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)

#plt.savefig("results/Figure5.pdf", format="pdf", bbox_inches="tight")